## The clear-cut version

In [1]:
def print_recommendations(knn_output):
    
    with open('../pkl_cellar/movie_list.pkl', 'rb') as file:
        movie_list = pickle.load(file)
    
    distances, indices = knn_output
    m_selection = []
    
    for i,res in enumerate(zip(distances.flatten(), indices.flatten())):
        distance, index = res
        name = movie_list.iloc[index]['title']
        if i == 0:
            or_movie = f'Users who liked {name} also like:'
        else:
            m_selection.append(name)
    
    return or_movie, m_selection

In [2]:
def movieid_vec(movie_id):
    
    with open('../pkl_cellar/movie_features.pkl', 'rb') as file:
        movie_features = pickle.load(file)
    
    return movie_features.loc[movie_id].values.reshape(1, -1)

In [3]:
def get_movieid(query):
    
    with open('../pkl_cellar/movie_map.pkl', 'rb') as file:
        movie_map = pickle.load(file)
    
    matches = []
    
    for title, movie_id in movie_map.items():
        
        # removing the year
        year_pattern = r'(.*)\s\(\d{4}\)$'
        extr_title, = re.findall(year_pattern, title)
        
        # get fuzz ration and remove CAsE-sEnsITiVity
        ratio = fuzz.ratio(extr_title.lower(), query)
        
        # ration must be more than 60 in order to match
        if ratio > 60:
            matches.append((extr_title, movie_id, ratio))
    
    if not matches:
        print('Oops, we couldn\'t find what you were looking for.')
        return
    
    # sorts the list of matches, with the highest ratio on top
    return sorted(matches, key=lambda x: x[2], reverse=True)[0][1]

In [4]:
def recommend_movies(query, n=5):
    
    with open('../pkl_cellar/model_knn.pkl', 'rb') as file:
        model_knn = pickle.load(file)
    
    movie_id = get_movieid(query)
    if not movie_id:
        return
    
    movie_vec = movieid_vec(movie_id)
    
    n_recs = n + 1
    recs = model_knn.kneighbors(movie_vec, n_neighbors=n_recs)
    
    return print_recommendations(recs)

### Checking if everything works

In [5]:
import pickle
import re
from fuzzywuzzy import fuzz

recommend_movies('walley')

('Users who liked WALL·E (2008) also like:',
 ['Up (2009)',
  'Iron Man (2008)',
  'Avatar (2009)',
  'Dark Knight, The (2008)',
  'Inception (2010)'])

### Checking output on frontend

In [6]:
def run_recommender(query):
    
    #if request.method == 'POST':
        #query = request.form['query']
        if recommend_movies(query, n=5) is None:
            error = "Oops! Try again..."
            return error
        else:
            info, recommendation = recommend_movies(query, n=5)

        #return render_template('index.html', output=recommendation)
        #return '{}{}'.format(info, recommendation)
        return recommendation

In [7]:
run_recommender('purge')

Oops, we couldn't find what you were looking for.


'Oops! Try again...'

In [8]:
type(run_recommender('toystory'))

list